### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and Facebook symptom data (loss of smell/taste, from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [28]:
#%load_ext autoreload
#%autoreload

In [29]:
#%reset

In [30]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [67]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])
    
    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    """
    print('f_jacob =')
    print(f_jacob)
    print('f_jacob[4,4] =', f_jacob[3, 3])
    """
    # 5x5
    # P_prior = f_jacob * P * f_jacob^T + Q
    P_prior = np.matmul(np.matmul(f_jacob, P), f_jacob.T) + Q
    
    """
    print()
    print('P prior - in predict step =')
    print(P_prior)
    print('P_prior[4,4] =', P_prior[3, 3])
    print()
    print()
    """
    return P_prior


def update_step(x_hat, x_hat_k1, P_prior, Rn, rho1, rho2, z_k, measure):
    # 5x5
    ep = 10**-8
    if measure is None:
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, 0, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    else:
                
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, rho1, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_prior * H^T + Rn
    Si = np.matmul(np.matmul(H, P_prior), H.T) + Rn

    # K = P_prior * H.T (H * P_prior * H.T + R)^-1
    # K_new = P_prior * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_prior, H.T), np.linalg.inv(Si))
    
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_post = P_prior - np.matmul(np.matmul(K_new, Si), K_new.T)


    return x_hat_k1_post, P_post, diff, K_new[3, 3]


def create_data_sets():
    seiir_fl = pd.read_csv(r'data/seiir_projections/florida_proj.csv', header=0,
                            index_col='date', parse_dates=True)

    seiir_ny = pd.read_csv(r'data/seiir_projections/new_york_proj.csv', header=0,
                            index_col='date', parse_dates=True)
    
    smell_data = data_sets.create_symptom_df()
    
    case_data = data_sets.create_case_df_county()
    
    return seiir_fl, seiir_ny, smell_data, case_data


def get_smell_data(fips, fb_data):

    if fips == 'New York City':
        nyc_fips = ['36005', '36061', '36047', '36085']
        fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
        fb_data_geo = fb_data_geo.mean(level='date')

        for borough in nyc_fips:
            fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')

    else:
        fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()

        # collapse down to a single index column (date)
        fb_data_geo.index = fb_data_geo.index.droplevel([0, 1])

    return fb_data_geo
    


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()


def calc_mse(prediction, actual):
    """
    Inputs should be two Pandas Series of same length.
    Outputs a float.
    """
    err = prediction - actual
    sum_sq_err = (err**2).sum()
    mse = sum_sq_err / err.count()
    return mse


def create_hh_data():
    # bring in new data
    full_data = pd.read_csv(r'data/from_challenge/overall-county.csv', header=0, dtype={'fips': 'str', 'pct_hh_cli': 'float64'},
                            parse_dates=[0])
    time_start = full_data['date'].min()
    time_end = full_data['date'].max()
    full_data.set_index(['fips', 'date'], inplace=True)
    full_data.sort_index(inplace=True)

    # derive count of survey respondents with household members having covid symptoms
    full_data['num_hh_cli'] = full_data['n'].mul(full_data['pct_hh_cli']/100.).round()
    full_data['num_hh_cli'] = full_data['num_hh_cli'].astype('int64')

    # group by county and date
    data_of_interest = full_data[['n', 'num_hh_cli']].copy().groupby(level=(0, 1)).sum()
    idx = pd.IndexSlice

    # create full date range
    date_rng = pd.date_range(time_start, time_end)
    iterables = [data_of_interest.index.levels[0], date_rng]
    new_index = pd.MultiIndex.from_product(iterables, names=['fips', 'date'])

    data_of_interest = data_of_interest.reindex(index=new_index)
    # this will have NaN values in the new index entries for which there was no
    # previous data -- fill them upon extracting a particular county
    
    return data_of_interest


In [32]:
def create_county_lists():
    case_data = data_sets.create_case_df_county()
    ny_counties = case_data[case_data['state'] == 'New York']['county'].unique()
    fl_counties = case_data[case_data['state'] == 'Florida']['county'].unique()
    return ny_counties, fl_counties

In [33]:
seiir_fl, seiir_ny, smell_data, case_data = create_data_sets()

hh_data = create_hh_data()

In [127]:
def run_county(the_state, county_name, seiir, smell_data, hh_data, case_data, measure='smell', K0=datetime.date(2020, 4, 18), delay=6):
    """
    Arguments:
        measure -- potential values are: 'smell', 'hh', or None
    """
    # set constants
    #K0 = datetime.date(2020, 4, 12)

    if county_name == 'New York City':
        the_county = county_name
    else:
        the_county = data_sets.get_fips(the_state, county_name)

    constants = {
        'alpha': 0.948786,
        'gamma1': 0.500000,
        'gamma2': 0.662215,
        'sigma': 0.266635,
        'theta': 6.000000
        }

    # set initial values for Kalman filter parameters
    P_mult = 1
    Q_mult = 1

    # Rn is the R noise covariance matrix; it remains constant thru the stepping of the
    # Kalman filter
    Rn_mult = 5*10**-8

    Rn_22 = 10000
    Rn_32 = 1000

    Rn_23 = 1000
    Rn_33 = 100

    Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0],
                             [0, 0, Rn_22, Rn_23, 0],
                             [0, 0, Rn_32, Rn_33, 0],
                             [0, 0, 0, 0, 0]])

    Q = Q_mult * np.eye(5)
    P = P_mult * np.eye(5)
    
    if the_county == 'New York City':
        county_pop = 0
        for each in ['36081', '36005', '36061', '36047', '36085']:
            this_count, state_pop = data_sets.get_pops(each)
            county_pop += this_count
    else:
        county_pop, state_pop = data_sets.get_pops(the_county)

    b = county_pop / state_pop

    # generate data
    case_data_geo = case_data.loc[the_county]['case_rate'].copy()
    smell_data_geo = get_smell_data(the_county, smell_data)
    

    if measure == 'hh':
        idx = pd.IndexSlice
        
        if the_county == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            hh_cli = hh_data.loc[idx['36081', :], :].loc['36081'].copy()
            hh_cli.fillna(method='pad', inplace=True)

            for borough in nyc_fips:
                bor_hh_cli = hh_data.loc[idx[borough, :], :].loc[borough].copy()
                bor_hh_cli.fillna(method='pad', inplace=True)
                hh_cli += bor_hh_cli

        else:
            hh_cli = hh_data.loc[idx[the_county, :], :].loc[the_county].copy()
            hh_cli.fillna(method='pad', inplace=True)

        # calculate moving averages on the fb and case data
        hh_cli_ma7 = hh_cli.rolling(window=7).mean()
        hh_cli_ma7 = hh_cli_ma7.iloc[6:, :]
        num_survey_ma7 = hh_cli_ma7['num_hh_cli']
        prop_cli_ma7 = num_survey_ma7.div(hh_cli_ma7['n'])
        
    elif measure == 'smell':
        prop_ma7, n_ma7, num_survey_ma7 = calc_fb_ma7(smell_data_geo)
        
    else:
        num_survey_ma7 = None


    case_ma7 = case_data_geo.rolling(window=7).mean()
    case_ma7_all = case_ma7.iloc[6:]
    

    # get starting compartment values for the state level
    x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

    # convert to the county level
    x_hat_k0 = b * x_hat_state_k0

    I2_county = x_hat_k0[3, 0]


    rho1 = .05
    rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


    # create empty dictionaries to hold the estimated values
    case_est = {}
    seiir_pred = {}

    diff_rat = {}

    K_val_dict = {}


    # Original data run ----------------
    start = K0
    k = start

    while k <= datetime.date(2020, 10, 23):    

    # each cycle of the while loop executes a step

        # get state level compartments
        x_hat_state_k, beta_k = get_predicts_prior(k, seiir)

        # step the state level compartments 1 day forward
        x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k, days=1)
        P = predict_step(x_hat_state_k, P, Q, beta_k, constants)
        
        # then 6 more
        P_est = P.copy()
        for i in range(6):
            x_hat_est = step_seiir(x_hat_state_k1, constants, beta_k, days=1)
            P_est = predict_step(x_hat_state_k1, P_est, Q, beta_k, constants)
            x_hat_state_k1 = x_hat_est.copy()

        # convert the state level compartments to county level values
        x_hat_k = b * x_hat_state_k
        x_hat_k1 = b * x_hat_state_k1

        
        indexDate = k + datetime.timedelta(days=7)
        # store seiir prediction before it's modified by Kalman filter
        seiir_pred[indexDate] = x_hat_k1[3, 0]

        # get measurements for current day
        if measure == 'smell':
            z_k = np.array([[0],
                            [0],
                            [prop_ma7.loc[k - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[k]],
                            [0]])    

        elif measure == 'hh':
            z_k = np.array([[0],
                            [0],
                            [prop_cli_ma7.loc[k - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[k]],
                            [0]])
            
        else:
            z_k = np.array([[0],
                            [0],
                            [0],
                            [case_ma7_all.loc[k]],
                            [0]])

        # predict step using the stepped fwd SEIIR compartment values 
        #P = predict_step(x_hat_k1, P, Q, beta_k, constants)

        # update step
        x_hat_post, P_post, the_diff, K_val = update_step(x_hat_k, x_hat_k1, P_est, Rn,
                                         rho1, rho2, z_k, measure)


        # store estimated values for proportion and case rate
        K_val_dict[indexDate] = K_val
        case_est[indexDate] = rho2 * x_hat_post[3, 0]


        diff_rat[indexDate] = the_diff[2, 0] / the_diff[3, 0]

        # update the P and k
        # comment out the P update because it has already been stepped once
        #P = P_post
        k += datetime.timedelta(days=1)

    # create pandas series of the estimated case rate
    predicted_case = pd.Series(case_est)
    predicted_seiir_prior = pd.Series(seiir_pred)
    K_val_series = pd.Series(K_val_dict)
    
    return predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_val_series

In [ ]:
predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('FL', 'Miami-Dade', seiir_fl, smell_data, hh_data, case_data, measure='smell', K0=datetime.date(2020, 4, 18), delay=0)

In [ ]:
K0 = datetime.date(2020, 4, 18)

ny_counties, fl_counties = create_county_lists()

In [ ]:

df_index = []
err_data = []

predictions_case_only = pd.DataFrame()
predictions_smell_0delay = pd.DataFrame()
predictions_smell_6delay = pd.DataFrame()
predictions_hh_0delay = pd.DataFrame()
predictions_hh_6delay = pd.DataFrame()

left = K0 + datetime.timedelta(days=7)
right = datetime.date(2020, 10, 23)
shifted_range = pd.date_range(start=left, end=right)
numLoop = 1

In [ ]:
for each in ny_counties:
    if each == 'New York City':
        fips = 'New York City'
    else:
        fips = data_sets.get_fips('NY', each)
        if (fips not in hh_data.index.levels[0]) or (fips not in smell_data.index.levels[1]):
            continue
    
    print('Starting', fips)
    
    county_data = {}
    
    conf_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure=None, K0=K0, delay=0)
        conf_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_confirmed-only'] = conf_mses.mean()
    county_data['ihme'] = calc_mse(predicted_seiir_prior.loc[left:right], case_ma7_all.loc[left:right])
    predictions_case_only[fips] = predicted_case
    
    """
    smell0_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=0)
        smell0_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_smell-0delay'] = smell0_mses.mean()
    predictions_smell_0delay[fips] = predicted_case
    

    smell6_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=6)
        smell6_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_smell-6delay'] = smell6_mses.mean()
    predictions_smell_6delay[fips] = predicted_case
    

    hh0_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=0)
        hh0_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_hh-0delay'] = hh0_mses.mean()
    predictions_hh_0delay[fips] = predicted_case
    

    hh6_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=6)
        hh6_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_hh-6delay'] = hh6_mses.mean()
    predictions_hh_6delay[fips] = predicted_case
    
    naive_pred = case_ma7_all.loc[left-datetime.timedelta(days=7):right-datetime.timedelta(days=7)].copy()
    naive_pred.index = shifted_range
    county_data['naive'] = calc_mse(naive_pred.loc[left:right], case_ma7_all.loc[left:right])
    """
    
    df_index.append(fips)
    err_data.append(county_data)
    print('Complete with', fips)

In [ ]:
err_ny_df = pd.DataFrame(err_data, index=df_index)

In [ ]:
err_ny_df.to_csv(r'output/err_ny_ihme.csv')

In [ ]:
predictions_case_only.to_csv(r'output/pred_ny_case_only.csv')
predictions_smell_0delay.to_csv(r'output/pred_ny_smell_0delay.csv')
predictions_smell_6delay.to_csv(r'output/pred_ny_smell_6delay.csv')
predictions_hh_0delay.to_csv(r'output/pred_ny_hh_0delay.csv')
predictions_hh_6delay.to_csv(r'output/pred_ny_hh_6delay.csv')

In [ ]:
df_index = []
err_data = []

predictions_case_only = pd.DataFrame()
predictions_smell_0delay = pd.DataFrame()
predictions_smell_6delay = pd.DataFrame()
predictions_hh_0delay = pd.DataFrame()
predictions_hh_6delay = pd.DataFrame()

left = K0 + datetime.timedelta(days=7)
right = datetime.date(2020, 10, 23)
shifted_range = pd.date_range(start=left, end=right)
numLoop = 1

In [ ]:
state_2L = 'FL'

for each in fl_counties:
    fips = data_sets.get_fips(state_2L, each)
    if (fips not in hh_data.index.levels[0]) or (fips not in smell_data.index.levels[1]):
        continue
    
    print('Starting', fips)
    
    county_data = {}
    
    conf_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2L, each, seiir_fl, smell_data, hh_data, case_data, measure=None, K0=K0, delay=0)
        conf_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['ihme'] = calc_mse(predicted_seiir_prior.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_confirmed-only'] = conf_mses.mean()
    predictions_case_only[fips] = predicted_case
    
    """
    smell0_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2L, each, seiir_fl, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=0)
        smell0_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_smell-0delay'] = smell0_mses.mean()
    predictions_smell_0delay[fips] = predicted_case
    

    smell6_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2L, each, seiir_fl, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=6)
        smell6_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_smell-6delay'] = smell6_mses.mean()
    predictions_smell_6delay[fips] = predicted_case
    

    hh0_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2L, each, seiir_fl, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=0)
        hh0_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_hh-0delay'] = hh0_mses.mean()
    predictions_hh_0delay[fips] = predicted_case
    

    hh6_mses = np.zeros(numLoop)
    for i in range(numLoop):
        predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2L, each, seiir_fl, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=6)
        hh6_mses[i] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    county_data['mse_hh-6delay'] = hh6_mses.mean()
    predictions_hh_6delay[fips] = predicted_case
    
    naive_pred = case_ma7_all.loc[left-datetime.timedelta(days=7):right-datetime.timedelta(days=7)].copy()
    naive_pred.index = shifted_range
    county_data['naive'] = calc_mse(naive_pred.loc[left:right], case_ma7_all.loc[left:right])
    """
    
    df_index.append(fips)
    err_data.append(county_data)
    print('Complete with', fips)


In [ ]:
err_fl_df = pd.DataFrame(err_data, index=df_index)

In [ ]:
err_fl_df.to_csv(r'output/err_fl_ihme.csv')

In [ ]:
predictions_case_only.to_csv(r'output/pred_fl_case_only.csv')
predictions_smell_0delay.to_csv(r'output/pred_fl_smell_0delay.csv')
predictions_smell_6delay.to_csv(r'output/pred_fl_smell_6delay.csv')
predictions_hh_0delay.to_csv(r'output/pred_fl_hh_0delay.csv')
predictions_hh_6delay.to_csv(r'output/pred_fl_hh_6delay.csv')

In [136]:

county = 'Erie'
state_2l = 'NY'
if state_2l == 'NY':
    the_seiir = seiir_ny
else:
    the_seiir = seiir_fl


county_preds = {}
survey_data = {}
estimates = ['confirmed_only', 'hh_delay0', 'smell_delay0']
measure_type = [None, 'hh', 'smell']
#delay_list = [0, 0, 0]
for i in range(len(estimates)):
    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2l, county, the_seiir, smell_data, hh_data, case_data, measure=measure_type[i], K0=datetime.date(2020, 4, 18), delay=0)
    survey_data[estimates[i]] = num_survey_ma7
    county_preds[estimates[i]] = predicted_case


In [65]:
#estimate_type = ['hh_delay6', 'confirmed_only', 'hh_delay0', 'smell_delay0', 'smell_delay6']
labels = ['Forecast using Case Count Only', 'Forecast Case Count w/ Household Symptoms', 'Forecast Case Count w/ Loss of Smell/Taste']

In [66]:
K0 = datetime.date(2020, 4, 18)
d = datetime.date(2020, 10, 23)
gold_right = d + datetime.timedelta(days=7)
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=K0, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

colors = ['blue', 'red', 'green']

for i in range(len(estimates)):
    plt.plot(county_preds[estimates[i]].index, county_preds[estimates[i]], label=labels[i], c=colors[i])
plt.plot(case_ma7_all.loc[K0:gold_right].index, case_ma7_all.loc[K0:gold_right], label='Actual Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior, label='IHME Forecast', c=)


#naieve estimate
naive_start = K0 + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[K0:d], label='Naive Estimate', c='orange')
plt.legend(loc='upper center')
plt.xticks(rotation=30, ha='right', rotation_mode='anchor')
#plt.ylim(2700, 3055)
#plt.xlim(datetime.date(2020, 7, 25), datetime.date(2020, 7, 30))


#plt.title(county)
plt.ylabel('Number of Cases per Day')

Text(0, 0.5, 'Number of Cases per Day')

In [ ]:
K_vals.head(50)

In [ ]:
plt.plot(K_vals.index, K_vals)
plt.title('Miami-Dade hh6 K[4, 4] Val')

In [ ]:
# error computations

In [ ]:
type(case_ma7_all)

In [ ]:
calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])

In [ ]:
test_df = pd.DataFrame()

In [ ]:
test_df['test'] = predicted_case

In [ ]:
test_df

In [ ]:
output_df = pd.DataFrame()
output_df[the_county] = predicted_case

In [ ]:
output_df.to_csv(r'output/predicted_case_example.csv')

# All plotting code below this

In [71]:
xlim_left = None
xlim_right = None

leftylim_low = None
leftylim_high = None

rightylim_low = None
rightylim_high = None

xtick_size = 14
xlabel_size = 14

In [72]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({'font.size': 20})
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 3
%matplotlib qt


In [73]:
K0 = datetime.date(2020, 4, 18)
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=K0, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]



In [152]:
### This is the one plot

end = d + datetime.timedelta(days=7)
d = datetime.date(2020, 10, 23)
fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[K0:end].index, case_ma7_all.loc[K0:end], label='Actual Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

#naive estimate
naive_start = K0 + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[K0:d], label='Naive Estimate', c='orange')

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(survey_data['hh_delay0'].loc[K0:end].index, survey_data['hh_delay0'].loc[K0:end], c='red',
         label='Count of Household Symptom Response', linewidth=width-1)
plt.plot(survey_data['smell_delay0'].loc[K0:end].index, survey_data['smell_delay0'].loc[K0:end], c='maroon',
         label='Count of Loss of Smell Response', linewidth=width-1)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Responses per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
#plt.plot(predicted_case.index, predicted_case.loc[K0:], label='Our 7-Day Forecast',
#         c=purple, linewidth=width)
plt.plot(county_preds['confirmed_only'].index, county_preds['confirmed_only'], c=purple,
         label='7-Day Forecast using Case Count Only', linewidth=width-1)
plt.plot(county_preds['hh_delay0'].index, county_preds['hh_delay0'], c='cyan',
         label='7-Day Forecast Case Count w/ Household Symptoms', linewidth=width-1)
plt.plot(county_preds['smell_delay0'].index, county_preds['smell_delay0'], c='green',
         label='7-Day Forecast Case Count w/ Loss of Smell/Taste', linewidth=width-1)




plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)

#plt.title('Miami-Dade, Household Symptoms, delay of 6 days')

(737517.0, 737711.0)

In [69]:
county_preds[estimates[0]]


2020-04-25    3288.680918
2020-04-26    3085.764595
2020-04-27    2970.918546
2020-04-28    2833.534226
2020-04-29    2195.126925
                 ...     
2020-10-26     641.448294
2020-10-27     620.286452
2020-10-28     678.196336
2020-10-29     682.448306
2020-10-30     676.258395
Length: 189, dtype: float64

In [70]:
K0

datetime.date(2020, 4, 18)

In [83]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -480
leftylim_high = 8000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [119]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -66
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 8

xtick_size = 14
xlabel_size = 14

In [105]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.5
rightylim_high = 17

xtick_size = 14
xlabel_size = 14

In [134]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -6
leftylim_high = 120

rightylim_low = -.125
rightylim_high = 3.8

xtick_size = 14
xlabel_size = 14

In [151]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 280

rightylim_low = -.8
rightylim_high = 22

xtick_size = 14
xlabel_size = 14